In [1]:
import pandas as pd
import numpy as np

# Sklearn's DecisionTree and Bagging Regressors


Ces deux algorithmes ont l'avantage d'accepter les valeurs manquantes. BaggingRegressor peut s'appliquer sur des DecisionTreeRegressors. On testera cela dans un deuxième temps.


## Préparation des données V1 - 2023/11/14


Stratégie :  
On remplace les heures par des floats, les identifiants de station par des entiers et on supprime la colonne 'date' et la colonne way.  
On a fait une division : 80% du dataset pour le train, 20% pour le test


- Import des données


In [2]:
path = "./../../data"
x_data = pd.read_csv(path + "/Xtrain_hgcGIrA.csv", sep=",")  # features
y_data = pd.read_csv(
    path + "/Ytrain_yL5OjS4.csv", sep=",", usecols=[1]
)  # occupancy rate

FileNotFoundError: [Errno 2] No such file or directory: './../data/Xtrain_hgcGIrA.csv'

- Mise en forme des données


In [ ]:
x_data = x_data.drop("date", axis=1)  # on supprime la colonne date
x_data = x_data.drop("way", axis=1)  # on supprime la colonne way
x_data["hour"] = x_data["hour"].apply(
    lambda x: int(x[:2]) if isinstance(x, str) else np.nan
)  # on transforme les strings des heures en float
# on tranforme les identifiants de gare en entiers
x_data["station"] = x_data["station"].astype("category")
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)

- Création d'un dataset de test et de validation


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x_data.values, y_data.values, train_size=0.8
)

## DecisionTreeRegressor - 2023/11/14 - Préparation des données V1


On utilise le regresseur et pas le classifieur car on ne cherche pas la classe de p0q0 mais sa valeur précise.


- Import de DecisionTreeRegressor


In [ ]:
from sklearn.tree import DecisionTreeRegressor

- Test sans jouer sur les paramètres


In [ ]:
DTR = DecisionTreeRegressor()
DTR.fit(X_train, y_train)
DTR.score(X_test, y_test)

0.7945943315174977

Le score est bien plus faible que pour XGBRegressor par exemple mais l'algorithme est moins performant.


On va chercher à optimiser les hyperparamètres suivants : max_depth, min_sample_split, criterion, splitter, max_features et random states.


- Choix de max_depth, min_sample_split


In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from time import time

start = time()

my_kfold = KFold(n_splits=5, shuffle=True, random_state=0)

max_depth = [x for x in range(175, 250, 1)]
min_samples_split = np.linspace(0.0001, 0.0015, 50)

tuned_parameters = {
    "max_depth": max_depth,
    "min_samples_split": min_samples_split,
}

DTR_params = GridSearchCV(
    DecisionTreeRegressor(), tuned_parameters, cv=my_kfold, n_jobs=-1
)

DTR_params.fit(X_train, y_train)
print(DTR_params.score(X_test, y_test), DTR_params.best_params_)
print(str(time() - start) + " sec")

0.7965467263124921 {'max_depth': 194, 'min_samples_split': 0.00035714285714285714}
46.13642692565918 sec


Premier test avec

- max_depth = [x for x in range(1,200, 10)]
- min_samples_split = np.linspace(0,1,20)

On obtient :

0.8074020105666642 {'max_depth': 191, 'min_samples_split': 0.05263157894736842}  
3.3284592628479004 sec  
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/model_selection/\_validation.py:425: FitFailedWarning:
100 fits failed out of a total of 2000.


Deuxième test avec :

- max_depth = [x for x in range(130,230, 5)]
- min_samples_split = np.linspace(0,0.1,20)

On obtient :

0.7796735574166819 {'max_depth': 135, 'min_samples_split': 0.05263157894736842}
4.773729085922241 sec
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/model_selection/\_validation.py:425: FitFailedWarning:
100 fits failed out of a total of 2000.


Troisième test avec :

- max_depth = [x for x in range(120,220, 1)]
- min_samples_split = np.linspace(0.001,0.8,50)

On obtient :

0.7970168047451205 {'max_depth': 203, 'min_samples_split': 0.001}  
43.490323543548584 sec

On n'a plus l'erreur car on a enlevé la valeur 0 du linspace. On obtient un score inférieur au deuxième test.
On fait un dernier test.


Quatrième test avec :

- max_depth = [x for x in range(175,250, 1)]
- min_samples_split = np.linspace(0.0001,0.0015,50)

On obtient :

0.7965467263124921 {'max_depth': 194, 'min_samples_split': 0.00035714285714285714}
46.13642692565918 sec


On choisit finalement les valeurs du quatrième test, mais on remarque que les meilleurs paramètres varient beaucoup d'un test à l'autre.


- Choix de criterion, splitter, max_features et random_states


Comme les GridSearchCV s'effectuent rapidement, on décide de tester tous les hyperparamètres restants à la fois.


In [ ]:
start = time()

my_kfold = KFold(n_splits=5, shuffle=True, random_state=0)

criterion = ["squared_error", "friedman_mse", "absolute_error", "poisson"]
splitter = ["best", "random"]
max_features = [x for x in range(5, 16, 1)] + [None]
random_state = [x for x in range(5, 16, 1)] + [None]

tuned_parameters = {
    "criterion": criterion,
    "splitter": splitter,
    "max_features": max_features,
    "random_state": random_state,
}

DTR_params = GridSearchCV(
    DecisionTreeRegressor(
        max_depth=194, min_samples_split=0.00035714285714285714
    ),
    tuned_parameters,
    cv=my_kfold,
    n_jobs=-1,
)

DTR_params.fit(X_train, y_train)
print(DTR_params.score(X_test, y_test), DTR_params.best_params_)
print(str(time() - start) + " sec")

0.9005365288430852 {'criterion': 'friedman_mse', 'max_features': 9, 'random_state': 11, 'splitter': 'best'}
6.230284214019775 sec


/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
3600 fits failed out of a total of 5760.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3600 fits failed with the following error:
Traceback (most recent call last):
  File "/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/tree/_classe

Premier test avec :

- criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
- splitter = ['best', 'random']
- max_features = [2,5,10,50,100,None]
- random_state = [2,5,10,50,100,None]

On obtient :

0.8723972336640182 {'criterion': 'friedman_mse', 'max_features': 10, 'random_state': None, 'splitter': 'best'}  
1.741943597793579 sec  
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/model_selection/\_validation.py:425: FitFailedWarning:
900 fits failed out of a total of 1440.


Deuxième test avec :

- criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
- splitter = ['best', 'random']
- max_features = [x for x in range(1,20,1)] + [None]
- random_state = [x for x in range(1,200,10)] + [None]

On obtient :

0.9005365288430852 {'criterion': 'friedman_mse', 'max_features': 9, 'random_state': 11, 'splitter': 'best'}  
15.812105417251587 sec

splitter et criterion semblent se fixer sur friedman_mse et best. Les valeurs de max_features et random_state fluctuent. On observe beaucoup d'échecs dans les fits, ce qui peut être un gros problème.


Troisième test avec :

- criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
- splitter = ['best', 'random']
- max_features = [x for x in range(5,16,1)] + [None]
- random_state = [x for x in range(5,16,1)] + [None]

On obtient :

0.9005365288430852 {'criterion': 'friedman_mse', 'max_features': 9, 'random_state': 11, 'splitter': 'best'}  
6.230284214019775 sec  
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/model_selection/\_validation.py:425: FitFailedWarning:
3600 fits failed out of a total of 5760.


On observe aucun changement.


On décide de se fixer sur les hyperparamètres suivants :

- max_depth = 194,
- min_sample_split = 0.00035714285714285714
- criterion = friedman_mse
- splitter = best (valeur par défaut)
- max_features = 9
- random_state = 11


## BaggingRegressor - 2023/11/14 - Préparation des données V1


Le bagging peut s'avérer assez efficace car il nous permet de diminier la variance dans l'entraînement. Or cette variance peut être importantes lorsqu'on a, comme ici, de nombreuses valeurs manquantes.


- import de BaggingRegressor


In [ ]:
from sklearn.ensemble import BaggingRegressor

- Test sans jouer sur les paramètres


In [ ]:
BR = (
    BaggingRegressor()
)  # l'estimateur par défaut est bien le DecisionTreeRegressor
BR.fit(X_train, y_train)
BR.score(X_test, y_test)

/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.8155241428815649

Le score de base est meilleur que pour le Decision Tree Regressor par défaut.


- choix de n_estimators par Grid Search CV


In [ ]:
start = time()

my_kfold = KFold(n_splits=5, shuffle=True, random_state=0)

n_estimators = range(1, 60, 1)

tuned_parameters = {"n_estimators": n_estimators}

BR_params = GridSearchCV(
    BaggingRegressor(), tuned_parameters, cv=my_kfold, n_jobs=-1
)

BR_params.fit(X_train, y_train)
print(BR_params.score(X_test, y_test), BR_params.best_params_)
print(str(time() - start) + " sec")

/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A

0.8248240623242661 {'n_estimators': 52}
16.026915311813354 sec


Premier test avec :

- n_estimators = range(1,200, 5)

On obtient :

0.8209945617844735 {'n_estimators': 31}  
34.43437886238098 sec


Deuxième test avec :

- n_estimators = range(1,100, 2)

On obtient :

0.8165741875395303 {'n_estimators': 23}  
21.16409945487976 sec


Troisième test avec :

- n_estimators = range(1,60, 1)

  0.8248240623242661 {'n_estimators': 52}  
  16.026915311813354 sec


- choix de n_estimators par OOB


In [ ]:
n_estimators = list(range(1, 90, 1))

start = time()
oob_error = []
for n in n_estimators:
    BR_oob_params = BaggingRegressor(n_estimators=n, oob_score=True, n_jobs=-1)
    BR_oob_params.fit(X_train, y_train)
    oob_error.append((BR_oob_params.oob_score_, n))
print(max(oob_error, key=lambda item: item[0]))
print(str(time() - start) + " sec")

/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:1261: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn(
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:1261: UserWarning: Some inputs do not have OOB scores. This probably means 

(0.8360753547264497, 45)
29.945748805999756 sec


Premier test avec :

- n_estimators = list(range(1,200, 5))

On obtient :

(0.8319470271477325, 26)  
20.519113540649414 sec

Attention, le score OOB ci-dessus correspond à un score sur le dataset de train et pas de test comme ci-dessus


Deuxième test avec :

- n_estimators = list(range(1,100, 2))

On obtient :

(0.8323998818965891, 69)
18.28619885444641 sec


Troisième test avec :

- n_estimators = list(range(1,90, 1))

On obtient :

(0.8360753547264497, 45)
29.945748805999756 sec

Comme avec GridSearchCV, n_estimators fluctue. On gardera le meilleur des deux techniques pour la suite.


- Test final en combinant nos résultats sur DecisionTreeRegressor


In [ ]:
BR_1 = BaggingRegressor(
    DecisionTreeRegressor(
        max_depth=194,
        min_samples_split=0.00035714285714285714,
        criterion="friedman_mse",
        max_features=9,
        random_state=11,
    ),
    n_estimators=52,
)
BR_1.fit(X_train, y_train)
BR_1.score(X_test, y_test)

/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.921884800812841

In [ ]:
BR_2 = BaggingRegressor(
    DecisionTreeRegressor(
        max_depth=194,
        min_samples_split=0.00035714285714285714,
        criterion="friedman_mse",
        max_features=9,
        random_state=11,
    ),
    n_estimators=45,
)
BR_2.fit(X_train, y_train)
BR_2.score(X_test, y_test)

/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.9182624859249583

On choisira BR_1.
